# **Maize Leaf Disease Classification Using VGG16**

# By 

 # **Salisu Yahaya**

# *salisuyahaya10@gmail.com**

In [1]:
import numpy as np  
import pandas as pd  

import os
print(os.listdir('/kaggle/input/corn-or-maize-leaf-disease-dataset/data/'))

# ** Libraries Importation**

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os 
import sys
import keras
from PIL import * 
import tensorflow as tf
from keras import optimizers
from keras import applications
from keras import backend as K
from os import listdir, makedirs
from keras.utils.data_utils import Sequence
from os.path import join, exists, expanduser
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input
from keras.optimizers import SGD, Adam
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model

In [3]:
epochs = 5

# **Function to Display Random Images from Each Class of The Dataset**

In [4]:
path = '/kaggle/input/corn-or-maize-leaf-disease-dataset/data/'
classes = os.listdir(path)

def display_four_class_images(random_number):
    for i in classes:
        new_path = path + i
        random_image = os.listdir(new_path)[random_number]
        print(new_path + '/' +random_image)
        plt.imshow(cv2.imread(new_path + '/' + random_image)[:,:,::-1])
        plt.show()

In [5]:
display_four_class_images(100)

In [6]:
im = cv2.imread('/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Gray_Leaf_Spot/Corn_Gray_Spot (404).JPG')
im.shape

In [7]:
im.dtype

In [8]:
im

# **Counting of Images in Each Class**

In [10]:
def dataset_size(path, classes):
    size = []
    for i in classes:
        size.append(len(os.listdir(path + i)))
        
    df = pd.DataFrame(columns = ['Type', 'No_of_Images'])
    df['Type'] = classes
    df['No_of_Images'] = size
    
    return df

In [11]:
dataset_size(path, classes)

# **Loading and checking the average image size of each class to make sure that there's no outlier shape in the Dataset.**

In [14]:
def average_image_size(label_name):
    r, g, b = [], [], []
    for image in os.listdir(path + label_name):
        im = cv2.imread(path + label_name+'/'+image)
        r.append(im.shape[0])
        g.append(im.shape[1])
        b.append(im.shape[2])
        
    return (sum(r)/len(r), sum(g)/len(g), sum(b)/len(b))

In [15]:
for label in classes:
    print(label, average_image_size(label))

# **Image(s) Extensions**

In [20]:
def image_extensions(label_name):
    extension = []
    for image in os.listdir(path + label_name):
        extension.append(image.split('.')[-1])
        
    return list(set(extension))

In [21]:
for label in classes:
    print(label, image_extensions(label))

In [24]:
train_data_generator =  ImageDataGenerator(rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                rescale=1.//255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                validation_split = 0.25)

In [70]:
# img = load_img(path+'/Blight/Corn_Blight (412).JPG')  # this is a PIL image
# x = img_to_array(img)  # this is a Numpy array  
# x = x.reshape((1,) + x.shape)  # this is a Numpy array  

# # the .flow() command below generates batches of randomly transformed images
# # and saves the results to the `preview/` directory
# i = 0
# for batch in train_data_generator.flow(x, batch_size=1,
#                           save_to_dir='/kaggle/input/preview', save_prefix='Corn_Blight', save_format='JPG'):
#     i += 1
#     if i > 20:
#         break  # otherwise the generator would loop indefinitely

In [26]:
train_dataset = train_data_generator.flow_from_directory(
                                    path,
                                    target_size = (224, 224),
                                    class_mode = "categorical",
                                    batch_size = 64,
                                    subset = "training")

val_dataset = train_data_generator.flow_from_directory(
                                    path,
                                    target_size = (224, 224),
                                    class_mode = "categorical",
                                    batch_size = 64,
                                    subset = "validation")

In [28]:
def visualize_datagenerator(no_of_images):
    for pic in range(no_of_images):
        image, label = val_dataset.next()
        print("image shape is:", image.shape)
        plt.imshow(image[0])
        print(image[20].shape)
        plt.show()

In [29]:
visualize_datagenerator(8)

In [33]:
input_layer = Input(shape = (224, 224, 3))

vgg_model = VGG16(include_top=False,weights="imagenet",classes=train_dataset.num_classes, input_shape = (224, 224, 3))

vgg16_model = Sequential()
vgg16_model.add(vgg_model)

vgg16_model.add(Flatten())
vgg16_model.add(Dense(256,activation='relu'))
vgg16_model.add(Dropout(0.25))
vgg16_model.add(Dense(4,activation='softmax'))

adam = Adam(lr= 0.0001, decay=0.0001 / epochs)


In [34]:
vgg16_model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [35]:
vgg16_model.summary()

In [36]:
plot_model(vgg16_model, to_file='vggmodel_plot.png', show_shapes=True, show_layer_names=True)

In [46]:
epochs = 20

In [47]:
history = vgg16_model.fit(train_dataset, epochs = epochs, validation_data = val_dataset, verbose = True)

In [48]:
history

In [49]:
history.history["loss"]

In [50]:
def visualize_training_epochs(v):
    plt.plot(v.history["accuracy"])
    plt.plot(v.history["val_accuracy"])
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("vgg16_model_accuracy")
    plt.legend(["Train", "Validation"])
    plt.show()
    
    plt.plot(v.history["loss"])
    plt.plot(v.history["val_loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("vgg16_model_loss")
    plt.legend(["Train", "Validation"])
    plt.show()

In [51]:
visualize_training_epochs(history)

In [52]:
vgg16_model.save("vgg16.h5")

In [53]:
val_dataset.class_indices

In [56]:
val_dataset.classes

In [57]:
len(val_dataset)

In [58]:
predictions = vgg16_model.predict_generator(val_dataset)
predictions

In [59]:
prediction_class = np.argmax(predictions, axis = 1)
prediction_class

In [60]:
len(prediction_class)

In [62]:
print(confusion_matrix(val_dataset.classes, prediction_class))

In [63]:
print(classification_report(val_dataset.classes, prediction_class))